In [2]:
#push what we are working on to github
#now that paths are working, pull dataloader into test model notebook and do the work there
#need to get under mse of 10
#correlation between training size and accuracy, more visuals for the final presentation



In [1]:
import os
from glob import glob
from warnings import simplefilter

In [2]:
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

#### Configure path parameters and read data files

In [3]:
ROOT_DIR = '/home/mids/m254572/Capstone'
IMAGES_DIR = os.path.join(ROOT_DIR, "data", "aircraft", "images")

RADOM_SEED = 2020
TRAIN_FRAC = 0.8
VAL_FRAC = 0.1

In [4]:
image_dir = IMAGES_DIR
labels_fp = os.path.join(ROOT_DIR, "data", "aircraft", "annotations.csv")
print(labels_fp)

/home/mids/m254572/Capstone/data/aircraft/annotations.csv


#### Load utility functions

In [5]:
os.chdir(ROOT_DIR)

In [6]:
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2

from src.utilities.data.aircraft_dataloader import get_dataloader

#### Define a minimal transformation pipeline

In [17]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transformations = A.Compose([
    A.Resize(256, 256),  # Ensure consistent size
    A.HorizontalFlip(p=0.5),  # Random horizontal flip
    A.RandomBrightnessContrast(p=0.5),  # Random brightness/contrast adjustment
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ToTensorV2(),  # Convert to tensor
])



#### Building a `torch.Dataloader`.

In [18]:
# Create the DataLoader
dataloader = get_dataloader(
  image_dir=image_dir,
  labels_fp=labels_fp,
  transformations=transformations,
  mode='train',
  train_frac=TRAIN_FRAC,
  val_frac=VAL_FRAC,
  seed=RADOM_SEED,
  batch_size=1,
  shuffle=True,
  num_workers=1,
)

#### Building a baseline MLP using `torch.nn.Module`

In [22]:

import torch
import torch.nn as nn


class BaselineMLP(nn.Module):
    def __init__(self):
        super(BaselineMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 256 * 3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(p=0.5)
        self.ln1 = nn.LayerNorm(128)  # Layer normalization after the first fully connected layer
        self.ln2 = nn.LayerNorm(64)   # Layer normalization after the second fully connected layer

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.ln1(x)  # Apply Layer Normalization
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.ln2(x)  # Apply Layer Normalization
        x = self.dropout(x)
        x = self.fc3(x)
        return x

#### Training the model

In [44]:
# Basic hyperparameters
learning_rate = 1
num_epochs = 10

In [45]:

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import OneCycleLR

baseline_mlp_model = BaselineMLP()
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.Adam(baseline_mlp_model.parameters(), lr=learning_rate)

scheduler = OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(dataloader), epochs=num_epochs)
# Training loop
for epoch in range(num_epochs):
  for i, (images, targets) in enumerate(dataloader):
    if i == len(dataloader) - 1: continue  # save the last batch for demonstration
    # Forward pass
    outputs = baseline_mlp_model(images)
    loss = criterion(outputs.squeeze(), targets)  # Ensure outputs are squeezed to match counts shape

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  scheduler.step()

  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 598.8333
Epoch [2/10], Loss: 396.6664
Epoch [3/10], Loss: 13.0013
Epoch [4/10], Loss: 2.7498
Epoch [5/10], Loss: 245.0599
Epoch [6/10], Loss: 66.4982
Epoch [7/10], Loss: 156.1909
Epoch [8/10], Loss: 85.2911
Epoch [9/10], Loss: 55.1097
Epoch [10/10], Loss: 8.2829


Check the model's prediction on the validation set for a `torch.Dataset` with the same `train_frac`, `val_frac`, and `seed`.

In [46]:
# Create the DataLoader
val_dataloader = get_dataloader(
  image_dir,
  labels_fp,
  transformations=transformations,
  mode='val',
  train_frac=TRAIN_FRAC,
  val_frac=VAL_FRAC,
  seed=RADOM_SEED,
  batch_size=1,
  shuffle=False,
  num_workers=1,
)

In [50]:
losses = []

for i, (val_images, targets) in enumerate(val_dataloader):
  predicted_counts = baseline_mlp_model(val_images)
  # We validate based on the mean absolute error
  losses.append(torch.abs(predicted_counts - targets).item())
  print(f"val image {i+1}, predicted count: {predicted_counts.item():.4f}, true count: {targets.item():.4f}")

mean_loss = sum(losses) / len(losses)
print(f"Mean absolute error: {mean_loss:.4f}")

val image 1, predicted count: 23.7112, true count: 31.0000
val image 2, predicted count: 21.1755, true count: 24.0000
val image 3, predicted count: 33.4970, true count: 19.0000
val image 4, predicted count: 58.2088, true count: 49.0000
val image 5, predicted count: 19.9987, true count: 27.0000
val image 6, predicted count: 23.1033, true count: 15.0000
val image 7, predicted count: 24.4448, true count: 30.0000
val image 8, predicted count: 36.7763, true count: 52.0000
val image 9, predicted count: 23.5959, true count: 39.0000
val image 10, predicted count: 25.5056, true count: 26.0000
Mean absolute error: 8.5601


In [ ]:
#Notes
#TEST 1 - BASIC MLP WITH LR OF 10-3 : ERROR OF 10.3
#TEST 2 - 

